In [ ]:
!pip install pandas
!pip install tqdm
!pip install numpy

In [ ]:
from transformers import AutoTokenizer
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
def get_text_split(text: str, tokenizer: AutoTokenizer, length: int = 250, overlap: int = 0, max_chunks: int = 200) -> list:
        """
        Função que divide o texto em pedaços de tamanho length com overlap de tamanho overlap.
        Parâmetros:
            text: texto a ser dividido
            length: tamanho de cada pedaço
            overlap: tamanho da sobreposição entre os pedaços
            max_chunks: número máximo de pedaços
        Retorno:
            l_total: lista com os pedaços do texto
        """
        l_total = []
        l_parcial = []
        n_words = len(text.split()) 
        #n_words = len(tokenizer.tokenize(text))
        n = n_words//(length-overlap)+1
        if n_words % (length-overlap) == 0:
            n = n-1
        if n ==0:
            n = 1
        n = min(n, max_chunks)
        for w in range(n):
            if w == 0:
                l_parcial = text.split()[:length]
            else:
                l_parcial = text.split()[w*(length-overlap):w*(length-overlap) + length]
            l = " ".join(l_parcial)
            if w==n-1:
                if len(l_parcial) < 0.75*length and n!=1:
                    continue
            l_total.append(l)
        return l_total

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
dados = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_gold_hab_atualizada.csv")
dados

In [ ]:
textos = []
for indice in tqdm(dados.index):
    textos.extend(get_text_split(dados['text'][indice], tokenizer, length=200, overlap=0, max_chunks=200))    

In [ ]:
dataframe = pd.DataFrame(textos, columns=['text'])
dataframe

In [ ]:
tqdm.pandas()
dataframe['tamanho'] = dataframe['text'].progress_apply(lambda x: len(tokenizer.tokenize(x)))

In [ ]:
dataframe

In [ ]:
dataframe['tamanho'].describe()

In [ ]:
dataframe = dataframe[dataframe['tamanho'] >= 10]
dataframe = dataframe[dataframe['tamanho'] <= 500]
dataframe

In [ ]:
dataframe = dataframe.select_dtypes(include=['object'])
dataframe.drop_duplicates(inplace=True)
dataframe.dropna(inplace=True)
dataframe.reset_index(drop=True, inplace=True)
dataframe

In [ ]:
dataframe.to_pickle("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/Indicios/dataset_gold_hab_atualizada_chunks.pkl")